# Machine Learning Engineer Nanodegree
## Supervised Learning
## Project 2: Building a Student Intervention System

Welcome to the second project of the Machine Learning Engineer Nanodegree! In this notebook, some template code has already been provided for you, and it will be your job to implement the additional functionality necessary to successfully complete this project. Sections that begin with **'Implementation'** in the header indicate that the following block of code will require additional functionality which you must provide. Instructions will be provided for each section and the specifics of the implementation are marked in the code block with a `'TODO'` statement. Please be sure to read the instructions carefully!

In addition to implementing code, there will be questions that you must answer which relate to the project and your implementation. Each section where you will answer a question is preceded by a **'Question X'** header. Carefully read each question and provide thorough answers in the following text boxes that begin with **'Answer:'**. Your project submission will be evaluated based on your answers to each of the questions and the implementation you provide.  

>**Note:** Code and Markdown cells can be executed using the **Shift + Enter** keyboard shortcut. In addition, Markdown cells can be edited by typically double-clicking the cell to enter edit mode.

### Question 1 - Classification vs. Regression
*Your goal for this project is to identify students who might need early intervention before they fail to graduate. Which type of supervised learning problem is this, classification or regression? Why?*

**Answer: **

This particular problem is a classification problem rather than a regression problem.  In classification, we are trying to classify unlabeled data points according to our predetermined categories, using data that is already labeled.  For regression problems, the goal is to estimate a value for specific feature on a continuous scale.  In this case, we are going to identify (or classify) each student as a potential pass or fail, which fits the definition discussed earlier.

## Exploring the Data
Run the code cell below to load necessary Python libraries and load the student data. Note that the last column from this dataset, `'passed'`, will be our target label (whether the student graduated or didn't graduate). All other columns are features about each student.

In [2]:
# Import libraries
import numpy as np
import pandas as pd
from time import time
from sklearn.metrics import f1_score

# Read student data
student_data = pd.read_csv("student-data.csv")
print "Student data read successfully!"

Student data read successfully!


### Implementation: Data Exploration
Let's begin by investigating the dataset to determine how many students we have information on, and learn about the graduation rate among these students. In the code cell below, you will need to compute the following:
- The total number of students, `n_students`.
- The total number of features for each student, `n_features`.
- The number of those students who passed, `n_passed`.
- The number of those students who failed, `n_failed`.
- The graduation rate of the class, `grad_rate`, in percent (%).


In [3]:
#print(student_data.head())

# TODO: Calculate number of students
n_students = student_data.shape[0]

# TODO: Calculate number of features
n_features = student_data.shape[1] - 1

# TODO: Calculate passing students
n_passed = student_data['passed'].eq('yes').sum()

# TODO: Calculate failing students
n_failed = student_data['passed'].eq('no').sum()

# TODO: Calculate graduation rate
grad_rate = float(n_passed*100)/n_students

# Print the results
print "Total number of students: {}".format(n_students)
print "Number of features: {}".format(n_features)
print "Number of students who passed: {}".format(n_passed)
print "Number of students who failed: {}".format(n_failed)
print "Graduation rate of the class: {:.2f}%".format(grad_rate)

Total number of students: 395
Number of features: 30
Number of students who passed: 265
Number of students who failed: 130
Graduation rate of the class: 67.09%


## Preparing the Data
In this section, we will prepare the data for modeling, training and testing.

### Identify feature and target columns
It is often the case that the data you obtain contains non-numeric features. This can be a problem, as most machine learning algorithms expect numeric data to perform computations with.

Run the code cell below to separate the student data into feature and target columns to see if any features are non-numeric.

In [4]:
# Extract feature columns
feature_cols = list(student_data.columns[:-1])

# Extract target column 'passed'
target_col = student_data.columns[-1]

# Show the list of columns
print "Feature columns:\n{}".format(feature_cols)
print "\nTarget column: {}".format(target_col)

# Separate the data into feature data and target data (X_all and y_all, respectively)
X_all = student_data[feature_cols]
y_all = student_data[target_col]

# Show the feature information by printing the first five rows
print "\nFeature values:"
print X_all.head()

Feature columns:
['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu', 'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']

Target column: passed

Feature values:
  school sex  age address famsize Pstatus  Medu  Fedu     Mjob      Fjob  \
0     GP   F   18       U     GT3       A     4     4  at_home   teacher   
1     GP   F   17       U     GT3       T     1     1  at_home     other   
2     GP   F   15       U     LE3       T     1     1  at_home     other   
3     GP   F   15       U     GT3       T     4     2   health  services   
4     GP   F   16       U     GT3       T     3     3    other     other   

    ...    higher internet  romantic  famrel  freetime goout Dalc Walc health  \
0   ...       yes       no        no       4         3     4    1    1      3   
1   ...       

### Preprocess Feature Columns

As you can see, there are several non-numeric columns that need to be converted! Many of them are simply `yes`/`no`, e.g. `internet`. These can be reasonably converted into `1`/`0` (binary) values.

Other columns, like `Mjob` and `Fjob`, have more than two values, and are known as _categorical variables_. The recommended way to handle such a column is to create as many columns as possible values (e.g. `Fjob_teacher`, `Fjob_other`, `Fjob_services`, etc.), and assign a `1` to one of them and `0` to all others.

These generated columns are sometimes called _dummy variables_, and we will use the [`pandas.get_dummies()`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html?highlight=get_dummies#pandas.get_dummies) function to perform this transformation. Run the code cell below to perform the preprocessing routine discussed in this section.

In [5]:
def preprocess_features(X):
    ''' Preprocesses the student data and converts non-numeric binary variables into
        binary (0/1) variables. Converts categorical variables into dummy variables. '''
    
    # Initialize new output DataFrame
    output = pd.DataFrame(index = X.index)

    # Investigate each feature column for the data
    for col, col_data in X.iteritems():
        
        # If data type is non-numeric, replace all yes/no values with 1/0
        if col_data.dtype == object:
            col_data = col_data.replace(['yes', 'no'], [1, 0])

        # If data type is categorical, convert to dummy variables
        if col_data.dtype == object:
            # Example: 'school' => 'school_GP' and 'school_MS'
            col_data = pd.get_dummies(col_data, prefix = col)  
        
        # Collect the revised columns
        output = output.join(col_data)
    
    return output

X_all = preprocess_features(X_all)
print "Processed feature columns ({} total features):\n{}".format(len(X_all.columns), list(X_all.columns))

Processed feature columns (48 total features):
['school_GP', 'school_MS', 'sex_F', 'sex_M', 'age', 'address_R', 'address_U', 'famsize_GT3', 'famsize_LE3', 'Pstatus_A', 'Pstatus_T', 'Medu', 'Fedu', 'Mjob_at_home', 'Mjob_health', 'Mjob_other', 'Mjob_services', 'Mjob_teacher', 'Fjob_at_home', 'Fjob_health', 'Fjob_other', 'Fjob_services', 'Fjob_teacher', 'reason_course', 'reason_home', 'reason_other', 'reason_reputation', 'guardian_father', 'guardian_mother', 'guardian_other', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']


### Implementation: Training and Testing Data Split
So far, we have converted all _categorical_ features into numeric values. For the next step, we split the data (both features and corresponding labels) into training and test sets. In the following code cell below, you will need to implement the following:
- Randomly shuffle and split the data (`X_all`, `y_all`) into training and testing subsets.
  - Use 300 training points (approximately 75%) and 95 testing points (approximately 25%).
  - Set a `random_state` for the function(s) you use, if provided.
  - Store the results in `X_train`, `X_test`, `y_train`, and `y_test`.

In [6]:
# TODO: Import any additional functionality you may need here
from sklearn.cross_validation import train_test_split

# TODO: Set the number of training points
num_train = 300

# Set the number of testing points
num_test = X_all.shape[0] - num_train

# TODO: Shuffle and split the dataset into the number of training and testing points above
X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, test_size=num_test, random_state=0)

# Show the results of the split
print "Training set has {} samples.".format(X_train.shape[0])
print "Testing set has {} samples.".format(X_test.shape[0])

Training set has 300 samples.
Testing set has 95 samples.


## Training and Evaluating Models
In this section, you will choose 3 supervised learning models that are appropriate for this problem and available in `scikit-learn`. You will first discuss the reasoning behind choosing these three models by considering what you know about the data and each model's strengths and weaknesses. You will then fit the model to varying sizes of training data (100 data points, 200 data points, and 300 data points) and measure the F<sub>1</sub> score. You will need to produce three tables (one for each model) that shows the training set size, training time, prediction time, F<sub>1</sub> score on the training set, and F<sub>1</sub> score on the testing set.

**The following supervised learning models are currently available in** [`scikit-learn`](http://scikit-learn.org/stable/supervised_learning.html) **that you may choose from:**
- Gaussian Naive Bayes (GaussianNB)
- Decision Trees
- Ensemble Methods (Bagging, AdaBoost, Random Forest, Gradient Boosting)
- K-Nearest Neighbors (KNeighbors)
- Stochastic Gradient Descent (SGDC)
- Support Vector Machines (SVM)
- Logistic Regression

### Question 2 - Model Application
*List three supervised learning models that are appropriate for this problem. For each model chosen*
- Describe one real-world application in industry where the model can be applied. *(You may need to do a small bit of research for this — give references!)* 
- What are the strengths of the model; when does it perform well? 
- What are the weaknesses of the model; when does it perform poorly?
- What makes this model a good candidate for the problem, given what you know about the data?

**Answer: **

In this project, we have a maximum of 300 observations in our training sample, with 48 binary features.  This is definitely not enough data per feature and will play a large part in our choice of algorithm, as mentioned during the discussion on the curse of dimensionality.  However, despite this lack of observations we might still be able to make valuable predictions using the following three algorithms:

**1)  Naive Bayes (BernoulliNB):**

GaussianNB has been used successfully in language processing and successful applications include spam filtering.  The following article titled "A Plan for Spam" illustrates how to apply the concepts of Bayes theorem for spam detection: http://www.paulgraham.com/spam.html.  BernoulliNB is similar in its theory and applications, also being used for document classification or spam filtering; it assumes a Bernoulli rather than Gaussian distribution for the conditional probability relationships.  Furthermore, the bernoulli distribution works well with binary features, such as the features that we have in this project.

Naive bayes has proven to be a powerful classfication system in practice.  It is relatively fast to train and less affected by irrelevant feature.  Also, it does not require extensive training before making good predictions and performs well in a wide variaty of samples.

One cause for concern with the algorithm is the assumption and choice of the distribution which requires intuition and domain knowledge to choose an appropriate one.  Naive bayes also assumes independence of the individual features and may fail to classify correctly when the correlation between the two features affects the classification; for instance, Naive bayes will be unable to predict the XOR relationship between two variables as mentioned in the notes.  In practice, we might still be able to make valuable predictions even in the presence of some dependence.

The data consists of all binary features with little training data per feature.  Naive bayes may be able to overcome this lack of data as it does not require significant training data as mentioned earlier.

**2)  Support Vector Machines (SVM):**

Support vector machines have been used successfully for classification in bioinformatics.  Some examples in literature can be found here: http://www.support-vector.net/bioinformatics.html.

The model allows a lot of flexibility in choosing the similarity/kernel, this allows us to use this for many different types of relationships.  It also performans well in high dimensions with a large number of features, even in the presence of small training sets.  Is easy to understand and visualize (in 2 dimensions) with an intuitive foundation of maximizing the margin between the two classes.  The model is a great choice for applications such as bioinformatics, where they are dealing with data in high dimensions with relatively low observations.

The flexibility of selecting a custom kernel can also be a weakness since it requires intuition and knowlege of the domain.  This may be easy in 2 dimensions but can be quite a challenge in higher dimensions.  SVM can also be sensitive to noise, resulting in overfitting to the noise and mislabled data points.  Finally, they are computationally expenseive, both in training and prediction, especially when working with large samples with high number of feature vectors.  As a result, if fast estimation and prediction is required, this is not the best algorithm.

In our case, we the SVM seems to be a good food especially considering we have a small sample size with a large number of features.  Choice of Kernel may be more difficult to select, but we shall stick with the default for now.

**3)  Ensemble Methods (AdaBoost):**

Ensemble methods, including AdaBoost, is being proposed as an effective algorithm for real time facial recognition.  This is discuessed in "Boosting for Fast Face Recognition" which can be found here:  http://www.face-rec.org/algorithms/boosting-ensemble/ratfg-rts01guo.pdf

Advantages of ensemble methods using weak learners include that it is very fast and simple.  Moreover, boosting can be used with any algorithm, even a combination of different algorithms (via voting).  More importantly, boosting using weak learners is resistent to overfitting.  As a result, it is a good fit for the online, fast face recognition technique mentioned above.

Some drawbacks include selection of the underlying algorithm and their respective parameters.  Also, if the learner is too complex then this can lead to overfitting.  Finally, the technique can be senstive to noise and outliers.  As a result, if there are many misclassified data points then this may not be the appropriate technique.

In this case, we shall be using the default DecisionTreeClassifier with a depth of 1 as our base estimator for the AdaBoost model.  This model tackles the problem from an entirely different perspective than the other two and complements our approach.  Finally, using a single level decision tree as the base estimator results in a relatively fast training algorithm that is linear in the number of observations (as opposed to SVM) and may be useful if computation time is an issue.


### Setup
Run the code cell below to initialize three helper functions which you can use for training and testing the three supervised learning models you've chosen above. The functions are as follows:
- `train_classifier` - takes as input a classifier and training data and fits the classifier to the data.
- `predict_labels` - takes as input a fit classifier, features, and a target labeling and makes predictions using the F<sub>1</sub> score.
- `train_predict` - takes as input a classifier, and the training and testing data, and performs `train_clasifier` and `predict_labels`.
 - This function will report the F<sub>1</sub> score for both the training and testing data separately.

In [7]:
def train_classifier(clf, X_train, y_train):
    ''' Fits a classifier to the training data. '''
    
    # Start the clock, train the classifier, then stop the clock
    start = time()
    clf.fit(X_train, y_train)
    end = time()
    
    # Print the results
    print "Trained model in {:.4f} seconds".format(end - start)

    
def predict_labels(clf, features, target):
    ''' Makes predictions using a fit classifier based on F1 score. '''
    
    # Start the clock, make predictions, then stop the clock
    start = time()
    y_pred = clf.predict(features)
    end = time()
    
    # Print and return results
    print "Made predictions in {:.4f} seconds.".format(end - start)
    return f1_score(target.values, y_pred, pos_label='yes')


def train_predict(clf, X_train, y_train, X_test, y_test):
    ''' Train and predict using a classifer based on F1 score. '''
    
    # Indicate the classifier and the training set size
    print "Training a {} using a training set size of {}. . .".format(clf.__class__.__name__, len(X_train))
    
    # Train the classifier
    train_classifier(clf, X_train, y_train)
    
    # Print the results of prediction for both training and testing
    print "F1 score for training set: {:.4f}.".format(predict_labels(clf, X_train, y_train))
    print "F1 score for test set: {:.4f}.".format(predict_labels(clf, X_test, y_test))

### Implementation: Model Performance Metrics
With the predefined functions above, you will now import the three supervised learning models of your choice and run the `train_predict` function for each one. Remember that you will need to train and predict on each classifier for three different training set sizes: 100, 200, and 300. Hence, you should expect to have 9 different outputs below — 3 for each model using the varying training set sizes. In the following code cell, you will need to implement the following:
- Import the three supervised learning models you've discussed in the previous section.
- Initialize the three models and store them in `clf_A`, `clf_B`, and `clf_C`.
 - Use a `random_state` for each model you use, if provided.
 - **Note:** Use the default settings for each model — you will tune one specific model in a later section.
- Create the different training set sizes to be used to train each model.
 - *Do not reshuffle and resplit the data! The new training points should be drawn from `X_train` and `y_train`.*
- Fit each model with each training set size and make predictions on the test set (9 in total).  
**Note:** Three tables are provided after the following code cell which can be used to store your results.

In [10]:
# TODO: Import the three supervised learning models from sklearn
from sklearn.naive_bayes import BernoulliNB
from sklearn.svm import SVC
from sklearn.ensemble import AdaBoostClassifier

# TODO: Initialize the three models
clf_A = BernoulliNB()
clf_B = SVC(random_state=0)
clf_C = AdaBoostClassifier(random_state=0)


# TODO: Set up the training set sizes
X_train_100 = X_train[:100]
y_train_100 = y_train[:100]

X_train_200 = X_train[:200]
y_train_200 = y_train[:200]

X_train_300 = X_train
y_train_300 = y_train

# TODO: Execute the 'train_predict' function for each classifier and each training set size
# train_predict(clf, X_train, y_train, X_test, y_test)

train_predict(clf_A, X_train_100, y_train_100, X_test, y_test)
print('')
train_predict(clf_A, X_train_200, y_train_200, X_test, y_test)
print('')
train_predict(clf_A, X_train_300, y_train_300, X_test, y_test)
print('')

train_predict(clf_B, X_train_100, y_train_100, X_test, y_test)
print('')
train_predict(clf_B, X_train_200, y_train_200, X_test, y_test)
print('')
train_predict(clf_B, X_train_300, y_train_300, X_test, y_test)
print('')

train_predict(clf_C, X_train_100, y_train_100, X_test, y_test)
print('')
train_predict(clf_C, X_train_200, y_train_200, X_test, y_test)
print('')
train_predict(clf_C, X_train_300, y_train_300, X_test, y_test)
print('')

Training a BernoulliNB using a training set size of 100. . .
Trained model in 0.0015 seconds
Made predictions in 0.0005 seconds.
F1 score for training set: 0.8382.
Made predictions in 0.0003 seconds.
F1 score for test set: 0.7612.

Training a BernoulliNB using a training set size of 200. . .
Trained model in 0.0011 seconds
Made predictions in 0.0003 seconds.
F1 score for training set: 0.8172.
Made predictions in 0.0002 seconds.
F1 score for test set: 0.7591.

Training a BernoulliNB using a training set size of 300. . .
Trained model in 0.0012 seconds
Made predictions in 0.0003 seconds.
F1 score for training set: 0.8082.
Made predictions in 0.0003 seconds.
F1 score for test set: 0.7714.

Training a SVC using a training set size of 100. . .
Trained model in 0.0011 seconds
Made predictions in 0.0009 seconds.
F1 score for training set: 0.8591.
Made predictions in 0.0008 seconds.
F1 score for test set: 0.7838.

Training a SVC using a training set size of 200. . .
Trained model in 0.0026 sec

### Tabular Results
Edit the cell below to see how a table can be designed in [Markdown](https://github.com/adam-p/markdown-here/wiki/Markdown-Cheatsheet#tables). You can record your results from above in the tables provided.

** Classifer 1 - BernoulliNB**  

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |        0.0015           |       0.0005           |    0.8382        |    0.7612       |
| 200               |        0.0011           |       0.0003           |    0.8172        |    0.7591       |
| 300               |        0.0012           |       0.0003           |    0.8082        |    0.7714       |

** Classifer 2 - SVC**  

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |       0.0011           |       0.0008          |      0.8591     |       0.7838    |
| 200               |     0.0026             |       0.0010           |     0.8693       |     0.7755      |
| 300               |      0.0053             |      0.0012            |    0.8692        |     0.7586     |

** Classifer 3 - AdaBoostClassifier**  

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |     0.0874             |      0.0042             |     0.9538          |     0.7200       |
| 200               |     0.0953               |    0.0035              |    0.8826        |    0.8058        |
| 300               |     0.1015               |     0.0044               |   0.8688          |  0.7794        |

## Choosing the Best Model
In this final section, you will choose from the three supervised learning models the *best* model to use on the student data. You will then perform a grid search optimization for the model over the entire training set (`X_train` and `y_train`) by tuning at least one parameter to improve upon the untuned model's F<sub>1</sub> score. 

### Question 3 - Choosing the Best Model
*Based on the experiments you performed earlier, in one to two paragraphs, explain to the board of supervisors what single model you chose as the best model. Which model is generally the most appropriate based on the available data, limited resources, cost, and performance?*

**Answer: **

All three models have their relative strengths and weaknesses.  The appropriate model in any scenario depends on the tradeoff between accuracy and time/computation constraints.  In this case, training and prediction time does not seem to be very important since we are working with data that has a very low time frequency, most likely annual observations.  Also, predictions are not needed in real time and it is more important for us to find the the most accurate model with the predictions being used for longer term decision making.  Moreover, even though the training set is expected to increase over time, the schools do not have the capacity to significantly increase the number of students and the training set can be expected to remain relatively small (i.e. won't number in the hundreds of thousands).  Finally, it is possible that the number of features increases in the future as we discover new possible variables that can impact the passing rate.

Given all of these points, the model that is best suited for this low observation, high dimension data set is the SVM model.  It has similar accuracy to the other training models but we can expect to improve this score after optimization of the parameters.  Unlike the others, we actually have 3 potential parameters to adjust.  Furthermore, the theory also supports the SVM framework for this type of a scenario.


### Question 4 - Model in Layman's Terms
*In one to two paragraphs, explain to the board of directors in layman's terms how the final model chosen is supposed to work. Be sure that you are describing the major qualities of the model, such as how the model is trained and how the model makes a prediction. Avoid using advanced mathematical or technical jargon, such as describing equations or discussing the algorithm implementation.*

**Answer: **

The SVM model is based on the simple concept of trying to find a line or border that divides objects in to different types.  For example, the SVM line on a chess board that is setup in the starting position would be the horizontal line in the middle of the board that is equally far away from the white and black pieces.  The reason we choose the middle line, is that it helps classify new points that may show up in to the different categories.  So in our chess example, if new pieces are added to the board the centre line would be better able to split the new pieces if they follow a similar pattern.  If the line was closer to the white pieces for instance, then a new piece that is just outside of this line would be labeled as black and based on the initial setup we would rather assume that things follow the existing order.  

The model is even more powerful than that simple example as the dividing line does not have to be a straight line.  Imagine a mall parking lot with the handicap or reserved parking spots being situated closer to the mall while the remainder of spots for the general public located further out.  The two types of parking spots can be separated by a circle and the SVM model is also capable of finding the circle that has the largest distance from both types of spots, this distance is also called 'margin'.  Ofcourse, in reality this can get quite complicated as we start considering more variables but the theory remains the same and the model is always trying to find this separator, whatever form it may take.  As more data points are added, they are classified based on what side of the separator they fall.  This is where finding the largest margin is helpful, as we can be more confident in classifying the new points.

### Implementation: Model Tuning
Fine tune the chosen model. Use grid search (`GridSearchCV`) with at least one important parameter tuned with at least 3 different values. You will need to use the entire training set for this. In the code cell below, you will need to implement the following:
- Import [`sklearn.grid_search.gridSearchCV`](http://scikit-learn.org/stable/modules/generated/sklearn.grid_search.GridSearchCV.html) and [`sklearn.metrics.make_scorer`](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.make_scorer.html).
- Create a dictionary of parameters you wish to tune for the chosen model.
 - Example: `parameters = {'parameter' : [list of values]}`.
- Initialize the classifier you've chosen and store it in `clf`.
- Create the F<sub>1</sub> scoring function using `make_scorer` and store it in `f1_scorer`.
 - Set the `pos_label` parameter to the correct value!
- Perform grid search on the classifier `clf` using `f1_scorer` as the scoring method, and store it in `grid_obj`.
- Fit the grid search object to the training data (`X_train`, `y_train`), and store it in `grid_obj`.

In [12]:
# TODO: Import 'GridSearchCV' and 'make_scorer'
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import make_scorer
from sklearn.cross_validation import KFold



# TODO: Create the parameters list you wish to tune
parameters = {'kernel':['linear','rbf'],'C':np.logspace(-3, 3, 7), 'gamma':np.logspace(-3, 3, 7)}

# Creating cross validation sets to improve estimates
cv_sets = KFold(X_train.shape[0], n_folds=3, shuffle=True, random_state = 0)


# TODO: Initialize the classifier
clf = SVC(random_state=0)

# TODO: Make an f1 scoring function using 'make_scorer' 
f1_scorer = make_scorer(f1_score, pos_label='yes')

# TODO: Perform grid search on the classifier using the f1_scorer as the scoring method
grid_obj = GridSearchCV(estimator=clf, param_grid=parameters, scoring=f1_scorer, cv=cv_sets)

# TODO: Fit the grid search object to the training data and find the optimal parameters
grid_obj = grid_obj.fit(X_train, y_train)

# Get the estimator
clf = grid_obj.best_estimator_

# Retrieve the best parameters
clfparams = grid_obj.best_params_

# Report the final F1 score for training and testing after parameter tuning
print "Tuned model has a training F1 score of {:.4f}.".format(predict_labels(clf, X_train, y_train))
print "Tuned model has a testing F1 score of {:.4f}.".format(predict_labels(clf, X_test, y_test))

Made predictions in 0.0053 seconds.
Tuned model has a training F1 score of 0.9717.
Made predictions in 0.0016 seconds.
Tuned model has a testing F1 score of 0.7919.


In [13]:
#See what the best parameters were (out of curiosity)
print(clfparams)

{'kernel': 'rbf', 'C': 1.0, 'gamma': 0.10000000000000001}


### Question 5 - Final F<sub>1</sub> Score
*What is the final model's F<sub>1</sub> score for training and testing? How does that score compare to the untuned model?*

**Answer: **

The optimized model has a training and testing F1 score of 0.9717 and 0.7919 respectively.  Both of these scores are higher than 0.8421 and 0.7826 for the training and testing scores from the initial untuned model.  The grid search selected a different kernel (rbf) from the initial linear model we tested and also selected a gamma of 0.1 over the default of 0.021 (1/# of features).  The smaller the gamma the more constrained the shape of the separator, so in this case the model is less constrained than the default values.  Overall, the final tuned model appears to be slightly better than the starting model and can be expected to improve as more training points are added.

> **Note**: Once you have completed all of the code implementations and successfully answered each question above, you may finalize your work by exporting the iPython Notebook as an HTML document. You can do this by using the menu above and navigating to  
**File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission.